In [5]:
import numpy as np
import numba
import cython
import torch

In [6]:
# Approach 1: NumPy only
def monte_carlo_numpy(nTrials):
    x = np.random.rand(nTrials)
    y = np.random.rand(nTrials)
    inside_circle = (x**2 + y**2) <= 1
    return 4 * np.sum(inside_circle) / nTrials

# Approach 2: NumPy + Numba
@numba.njit
def monte_carlo_numpy_numba(nTrials):
    x = np.random.rand(nTrials)
    y = np.random.rand(nTrials)
    inside_circle = (x**2 + y**2) <= 1
    return 4 * np.sum(inside_circle) / nTrials

# Approach 3: Cython
# (Compile it using: python3 setup.py build_ext --inplace)
import montecarlopy as mc

# Approach 4: PyTorch with GPU
def monte_carlo_pytorch(nTrials):
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("GPU not available, using CPU.")

    XrandCoords = torch.rand(nTrials, device=device) * 2 - 1
    YrandCoords = torch.rand(nTrials, device=device) * 2 - 1
    inside_circle = (XrandCoords**2 + YrandCoords**2) <= 1
    return 4 * torch.sum(inside_circle).item() / nTrials

In [10]:
# Number of trials
nTrials = int(1E7)

# Timing the execution
print("Timing NumPy only...")
%time pi_numpy = monte_carlo_numpy(nTrials)

print("\nTiming NumPy + Numba...")
%time pi_numpy_numba = monte_carlo_numpy_numba(nTrials)

# Uncomment the next line after compiling the Cython module
# from monte_carlo_cython import monte_carlo_cython
print("\nTiming Cython...")
%time pi_cython = mc.monte_carlo_cython(nTrials)

print("\nTiming PyTorch...")
%time pi_pytorch = monte_carlo_pytorch(nTrials)
# Print the results
print("\nResults:")
print(f"Pi (NumPy only): {pi_numpy}")
print(f"Pi (NumPy + Numba): {pi_numpy_numba}")
print(f"Pi (Cython): {pi_cython}")
print(f"Pi (PyTorch): {pi_pytorch}")

Timing NumPy only...
CPU times: total: 125 ms
Wall time: 281 ms

Timing NumPy + Numba...
CPU times: total: 93.8 ms
Wall time: 173 ms

Timing Cython...
CPU times: total: 3.2 s
Wall time: 5.91 s

Timing PyTorch...
Using GPU: NVIDIA GeForce GTX 1660 Ti
CPU times: total: 156 ms
Wall time: 211 ms

Results:
Pi (NumPy only): 3.1412172
Pi (NumPy + Numba): 3.1414684
Pi (Cython): 3.1409968
Pi (PyTorch): 3.141142


In [8]:
torch.cuda.empty_cache()